<a href="https://colab.research.google.com/github/ajitonelsonn/TELE_Feedback/blob/main/Llama_Chatbot_with_Sentiment_Analysis_Integration_TL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TELE Feedback**

TELE Feedback is an AI-powered chatbot designed for a telecommunications company in Timor-Leste. It allows customers to provide feedback while integrating sentiment analysis to assess the tone of their messages. The project includes a simple interface for interaction and visualization of sentiment trends.

## 1. **Installing Required Libraries**

This cell installs all the libraries that the project depends on. These libraries are necessary for different functionalities like interacting with the Hugging Face models, creating a web interface for the chatbot, handling database operations, and visualizing feedback data.

- **`huggingface_hub`**: Allows the project to access pre-trained models from Hugging Face's model hub.
- **`gradio`**: Used to create an interactive web interface for the chatbot where users can input text and get responses.
- **`transformers`**: A library from Hugging Face that provides tools to use pre-trained models for natural language processing tasks, like language generation and sentiment analysis.
- **`sqlalchemy`**: An SQL toolkit for Python that helps in managing the storage and retrieval of data in a database.
- **`pandas`**: Used for data manipulation and analysis.
- **`matplotlib`**: A popular plotting library used for visualizing data, such as feedback trends.
- **`torch`**: PyTorch is a framework used for loading and running machine learning models.

In [ ]:
!pip install huggingface_hub gradio git+https://github.com/huggingface/transformers sqlalchemy pandas matplotlib torch

## 2. **Importing Necessary Libraries**

This cell imports all the necessary libraries and modules required throughout the project.

- **`huggingface_hub.login`**: Used to log in to Hugging Face, so you can access models stored in your account or publicly available models.
- **`gradio`**: Used for creating a user interface where people can interact with your chatbot.
- **`transformers.AutoModelForCausalLM` and `AutoTokenizer`**: These two classes load a pre-trained language model and its tokenizer, which are essential for generating chatbot responses.
- **SQLAlchemy-related imports**:
  - **`create_engine`**: Connects the code to the SQLite database.
  - **`Column`, `Integer`, `String`**: These are data types used to define the database schema.
  - **`declarative_base`**: A base class used to define the database table structure.
  - **`sessionmaker`**: Creates sessions to interact with the database.
- **`torch`**: PyTorch framework, required to load and run machine learning models like the chatbot.
- **`matplotlib.pyplot`**: This is used to plot graphs or charts, such as visualizing feedback data trends.
- **`collections.Counter`**: This utility helps in counting the occurrences of feedback sentiments (positive, negative, neutral).

In [ ]:
from huggingface_hub import login
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import torch
import matplotlib.pyplot as plt
from collections import Counter

## 3. **Hugging Face Login**

#### Code:
```python
# Hugging Face login (you need to provide your token)
login(token="your_huggingface_token_here")
```

#### Explanation:
This cell is responsible for logging into Hugging Face using your personal API token. If you don’t log in, you won’t be able to access models that are either private or require authentication.

- **`login(token="your_huggingface_token_here")`**: Replace `"your_huggingface_token_here"` with your actual token from Hugging Face. This allows the script to access models and datasets from the Hugging Face platform.

---

In [ ]:
# Hugging Face login (you need to provide your token)
login(token="")



## 4. **Loading the Pre-Trained Model and Tokenizer**

#### Code:
```python
# Load pre-trained model and tokenizer from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("model_name")
model = AutoModelForCausalLM.from_pretrained("model_name")
```

#### Explanation:
This cell loads the language model and tokenizer that will be used to generate chatbot responses.

- **`AutoTokenizer.from_pretrained("model_name")`**: This loads a tokenizer from Hugging Face. The tokenizer processes raw text input and converts it into tokens that the model can understand.
- **`AutoModelForCausalLM.from_pretrained("model_name")`**: This loads the pre-trained language model. A causal language model (like GPT) is capable of generating text responses based on the input tokens.

The `model_name` should be replaced with the actual name of the pre-trained model you are using (for example, "gpt-3", "gpt-2", etc.).

## 5. **Creating the Chatbot Function**

#### Code:
```python
def chatbot(input_text):
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"])
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response
```

#### Explanation:
This function is the core of the chatbot logic. It takes user input, generates a response, and returns it.

- **`input_text`**: The input text that the user provides.
- **`tokenizer(input_text, return_tensors="pt")`**: Tokenizes the user input and converts it into a tensor format required by the model.
- **`model.generate(inputs["input_ids"])`**: The pre-trained model generates a response based on the input text tokens.
- **`tokenizer.decode(outputs[0], skip_special_tokens=True)`**: Converts the output tokens from the model back into human-readable text, skipping any special tokens like `<pad>` or `<eos>`.
- **`return response`**: The chatbot returns the generated response.


## 6. **Creating the Gradio Interface for the Chatbot**

#### Code:
```python
# Create Gradio interface for the chatbot
interface = gr.Interface(fn=chatbot, inputs="text", outputs="text")
interface.launch()
```

#### Explanation:
This cell creates and launches a Gradio interface where users can interact with the chatbot.

- **`gr.Interface()`**: This creates a simple web interface where users can input text and get a response. It takes three arguments:
  - **`fn=chatbot`**: Specifies the function (chatbot) to be executed when the user provides input.
  - **`inputs="text"`**: Specifies the input type for the interface, which is text in this case.
  - **`outputs="text"`**: Specifies the output type, which will be a text response.
- **`interface.launch()`**: This launches the web interface, making it accessible in a web browser where users can start chatting with the bot.

## 7. **Loading Sentiment Analysis Pipeline**

#### Code:
```python
from transformers import pipeline

# Load sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis")
```

#### Explanation:
This cell loads a pre-trained sentiment analysis model from Hugging Face, which will be used to analyze the sentiment of user input.

- **`pipeline("sentiment-analysis")`**: This creates a sentiment analysis pipeline. The pipeline will classify text as **positive**, **neutral**, or **negative** based on its content. The model used here is a pre-trained one provided by Hugging Face.

## 8. **Creating the Chatbot with Sentiment Analysis Function**

#### Code:
```python
def chatbot_with_sentiment(input_text):
    # Get chatbot response
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"])
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Analyze sentiment of user input
    sentiment = sentiment_analyzer(input_text)
    return response, sentiment
```

#### Explanation:
This function generates both a chatbot response and performs sentiment analysis on the user’s input.

- **`tokenizer(input_text, return_tensors="pt")`** and **`model.generate()`**: The same steps as the `chatbot()` function to generate the chatbot response.
- **`sentiment_analyzer(input_text)`**: The sentiment analysis model analyzes the user's input and returns whether the input was positive, negative, or neutral.
- **`return response, sentiment`**: The function returns both the chatbot's response and the sentiment analysis result.

## 9. **Creating Gradio Interface for Chatbot with Sentiment Analysis**

#### Code:
```python
# Create Gradio interface for the chatbot with sentiment analysis
interface = gr.Interface(fn=chatbot_with_sentiment, inputs="text", outputs=["text", "label"])
interface.launch()
```

#### Explanation:
This creates a Gradio interface that not only generates a chatbot response but also shows the sentiment of the user's input.

- **`fn=chatbot_with_sentiment`**: The function to be executed is `chatbot_with_sentiment`, which returns both a chatbot response and sentiment analysis.
- **`inputs="text"`**: The input type is text.
- **`outputs=["text", "label"]`**: The output has two parts:
  - **Text**: The chatbot's response.
  - **Label**: The sentiment analysis result (positive, negative, or neutral).
- **`interface.launch()`**: This launches the web interface.

## 10. **Defining the SQLAlchemy Model for Feedback**

#### Code:
```python
# Define the SQLAlchemy model for storing feedback
Base = declarative_base()

class Feedback(Base):
    __tablename__ = 'feedback

'
    id = Column(Integer, primary_key=True)
    user_input = Column(String)
    chatbot_response = Column(String)
    sentiment = Column(String)
```

#### Explanation:
This cell defines a **SQLAlchemy** model to store feedback data from the chatbot interaction. The data includes the user’s input, the chatbot’s response, and the sentiment of the user’s input.

- **`Base = declarative_base()`**: Sets up a base class from which all database models (tables) will inherit.
- **`Feedback` class**: Defines a table named `feedback` with columns:
  - **`id`**: The primary key, a unique integer identifier for each row.
  - **`user_input`**: Stores the user’s input.
  - **`chatbot_response`**: Stores the chatbot’s generated response.
  - **`sentiment`**: Stores the sentiment (positive, negative, or neutral) of the user’s input.

## 11. **Connecting to the SQLite Database**

#### Code:
```python
# Connect to the SQLite database
engine = create_engine('sqlite:///feedback.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()
```

#### Explanation:
This sets up the connection to an SQLite database called `feedback.db`.

- **`create_engine('sqlite:///feedback.db')`**: Creates a connection to a SQLite database file named `feedback.db`. If it doesn’t exist, it will be created.
- **`Base.metadata.create_all(engine)`**: Creates all the tables defined by the model (in this case, the `Feedback` table) in the database.
- **`sessionmaker(bind=engine)`**: Creates a session factory bound to the database engine, which allows interactions with the database.
- **`session = Session()`**: This is the session object used to execute transactions with the database (e.g., inserting or retrieving feedback data).

##12. **Function to Save Feedback in the Database**

#### Code:
```python
# Function to save feedback to the database
def save_feedback(user_input, chatbot_response, sentiment):
    new_feedback = Feedback(user_input=user_input, chatbot_response=chatbot_response, sentiment=sentiment)
    session.add(new_feedback)
    session.commit()
```

#### Explanation:
This function is responsible for saving the feedback data (user input, chatbot response, sentiment) into the database.

- **`save_feedback()`**: This function takes three arguments:
  - **`user_input`**: The user's input text.
  - **`chatbot_response`**: The chatbot's response to the input.
  - **`sentiment`**: The sentiment of the user's input.
- **`new_feedback = Feedback(...)`**: Creates a new entry (row) in the `Feedback` table with the provided data.
- **`session.add(new_feedback)`**: Adds the new entry to the session.
- **`session.commit()`**: Commits the transaction, saving the new feedback entry into the database.


## 13. **Gradio Interface for Displaying Data**

#### Code:
```python
import pandas as pd

# Define a function to retrieve and show feedback data
def show_feedback():
    feedback_data = session.query(Feedback).all()
    feedback_dict = {"User Input": [fb.user_input for fb in feedback_data],
                     "Chatbot Response": [fb.chatbot_response for fb in feedback_data],
                     "Sentiment": [fb.sentiment for fb in feedback_data]}
    return pd.DataFrame(feedback_dict)

# Create Gradio interface to show feedback
data_interface = gr.Interface(fn=show_feedback, inputs=None, outputs="dataframe")
data_interface.launch()
```

#### Explanation:
This part of the code creates a new Gradio interface that displays stored feedback data from the database in a table format.

- **`show_feedback()`**:
  - This function retrieves all feedback entries from the `Feedback` table.
  - The feedback entries are converted into a dictionary format that contains lists of user input, chatbot responses, and sentiments.
  - The dictionary is converted into a Pandas DataFrame, which is a tabular structure that Gradio can display.
  
- **`gr.Interface(fn=show_feedback, inputs=None, outputs="dataframe")`**: Creates a Gradio interface for displaying the feedback data as a table. No user input is needed, and the output is a DataFrame (a table of data).

- **`data_interface.launch()`**: Launches the web interface to display feedback data.



## 14. **Plotting Feedback Sentiment Trends**

#### Code:
```python
def plot_sentiment_trends():
    feedback_data = session.query(Feedback).all()
    sentiments = [fb.sentiment for fb in feedback_data]
    sentiment_counts = Counter(sentiments)
    
    # Plot the sentiment counts
    plt.bar(sentiment_counts.keys(), sentiment_counts.values())
    plt.xlabel('Sentiment')
    plt.ylabel('Count')
    plt.title('Sentiment Distribution')
    plt.show()

# Create Gradio interface to show sentiment trends
plot_interface = gr.Interface(fn=plot_sentiment_trends, inputs=None, outputs="plot")
plot_interface.launch()
```

#### Explanation:
This part creates a visualization of the sentiment trends from the feedback data.

- **`plot_sentiment_trends()`**:
  - Retrieves the sentiment data from all feedback entries.
  - Counts the occurrences of each sentiment (positive, negative, neutral) using `Counter`.
  - Plots a bar chart showing the distribution of sentiments.

- **`plt.bar()`**: Creates a bar chart with sentiment categories on the x-axis and their counts on the y-axis.
- **`gr.Interface(fn=plot_sentiment_trends, inputs=None, outputs="plot")`**: Creates a Gradio interface that displays a plot. No input is required, and the output is a plot.
- **`plot_interface.launch()`**: Launches the interface so users can see the sentiment trends.

---

## 15. **Saving Feedback after Each Chat**

#### Code:
```python
def chatbot_with_sentiment_and_save(input_text):
    # Get chatbot response
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"])
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Analyze sentiment
    sentiment = sentiment_analyzer(input_text)
    
    # Save feedback to the database
    save_feedback(user_input=input_text, chatbot_response=response, sentiment=sentiment[0]['label'])
    
    return response, sentiment[0]['label']

# Gradio interface for chatbot with sentiment analysis and feedback saving
interface = gr.Interface(fn=chatbot_with_sentiment_and_save, inputs="text", outputs=["text", "label"])
interface.launch()
```

#### Explanation:
This final part integrates everything: chatbot functionality, sentiment analysis, and saving feedback.

- **`chatbot_with_sentiment_and_save()`**:
  - Generates a chatbot response.
  - Analyzes the sentiment of the user’s input.
  - Saves the user input, chatbot response, and sentiment into the database by calling `save_feedback()`.
  
- **`gr.Interface()`**: Creates a Gradio interface that handles chatbot interactions, sentiment analysis, and automatically saves the feedback.
  
- **`interface.launch()`**: Launches the chatbot interface where users can interact and have their feedback stored.

In [ ]:
# Database setup
Base = declarative_base()
engine = create_engine('sqlite:///telecom_support.db', echo=False)
Session = sessionmaker(bind=engine)

class Feedback(Base):
    __tablename__ = 'feedback'
    id = Column(Integer, primary_key=True)
    category = Column(String)
    original_text = Column(String)
    sentiment = Column(String)
    response = Column(String)

Base.metadata.create_all(engine)

In [ ]:
# Load Llama 2 model
model_id = "NousResearch/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# Updated categories for a telecom company
CATEGORIES = ["Mobile Network", "Mobile Package", "Mosan Emoney", "Recharge"]

In [ ]:
def generate_response(prompt, max_length=500):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1, temperature=0.7)

    # Decode the generated response
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
def process_feedback(message, category):
    prompt = f"""
    You are a professional customer service assistant for a telecom company. A customer has submitted the following feedback in the category of {category}:

    "{message}"

    Please respond with:
    1. A clear identification of the sentiment (positive, negative, or neutral).
    2. A concise, empathetic, and professional response addressing the customer's concern.
    3. If appropriate, include a solution or next steps for the customer.

    Format the output as:
    Sentiment: [positive/negative/neutral]
    Response: [Your customer response]

    No extra information or format explanations should be included.
    """

    # Generate response from the model
    response = generate_response(prompt, max_length=500)

    return response

In [ ]:
def chatbot(message, category):
    if not message.strip():
        return "Error: Please provide feedback text."
    if not category:
        return "Error: Please select a category."

    try:
        # Process the feedback with the category
        result = process_feedback(message, category)

        # Parse the result
        lines = result.split('\n')
        parsed_result = {}
        for line in lines:
            if ':' in line:
                key, value = line.split(':', 1)
                parsed_result[key.strip()] = value.strip()

        # Extract sentiment and response
        sentiment = parsed_result.get('Sentiment', 'Unknown')
        response = parsed_result.get('Response', '')

        # If no response is generated, use a fallback
        if not response:
            fallback_prompt = f"Provide a brief, helpful response to a customer's feedback about {category}: '{message}'"
            response = generate_response(fallback_prompt, max_length=150)

        # Save the feedback to the database
        session = Session()
        feedback = Feedback(
            category=category,
            original_text=message,
            sentiment=sentiment,
            response=response
        )
        session.add(feedback)
        session.commit()

        return response

    except Exception as e:
        error_prompt = f"Briefly apologize and offer assistance for an error in processing feedback about {category}: '{message}'"
        error_response = generate_response(error_prompt, max_length=100)
        return error_response

In [ ]:
# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Telecommunication Customer Support Chatbot")

    category = gr.Dropdown(choices=CATEGORIES, label="Select Feedback Category")
    message = gr.Textbox(lines=3, placeholder="Type your feedback here...", label="Your Feedback")

    # Use TextArea for better long-text handling
    output = gr.TextArea(label="Chatbot Response", lines=10)

    submit_btn = gr.Button("Submit Feedback")
    submit_btn.click(fn=chatbot, inputs=[message, category], outputs=output)

    gr.Examples(
        examples=[
            ["Mobile Network", "The network is down in my area for days."],
            ["Mobile Package", "I'm happy with the new data package!"],
            ["Mosan Emoney", "I have an issue with my last Mosan Emoney transfer."],
            ["Recharge", "I need help with my recent recharge transaction."]
        ],
        inputs=[category, message]
    )

In [ ]:
demo.launch()

In [ ]:
# Function to plot the data
def plot_sentiment_distribution():
    # Query the database to retrieve the feedback data
    session = Session()
    feedback_data = session.query(Feedback).all()

    # Processing data: count sentiments per category
    category_sentiment_count = {}

    for feedback in feedback_data:
        category = feedback.category
        sentiment = feedback.sentiment.lower()  # Convert sentiment to lowercase for consistency

        if category not in category_sentiment_count:
            category_sentiment_count[category] = Counter()

        category_sentiment_count[category][sentiment] += 1

    session.close()

    # Prepare data for plotting
    categories = list(category_sentiment_count.keys())
    positive_counts = [category_sentiment_count[cat].get('positive', 0) for cat in categories]
    negative_counts = [category_sentiment_count[cat].get('negative', 0) for cat in categories]
    neutral_counts = [category_sentiment_count[cat].get('neutral', 0) for cat in categories]

    # Plotting the data with adjusted bar width and position
    x = range(len(categories))  # x-coordinates
    bar_width = 0.2  # Adjust the bar width

    plt.figure(figsize=(10, 6))

    # Plotting positive, negative, and neutral sentiment bars for each category
    plt.bar([i - bar_width for i in x], positive_counts, width=bar_width, label='Positive', align='center')
    plt.bar(x, negative_counts, width=bar_width, label='Negative', align='center')
    plt.bar([i + bar_width for i in x], neutral_counts, width=bar_width, label='Neutral', align='center')

    # Customizing the plot
    plt.xlabel('Feedback Categories')
    plt.ylabel('Number of Feedback')
    plt.title('Sentiment Distribution by Feedback Categories')
    plt.xticks(x, categories, rotation=45)
    plt.legend()

    # Save the plot as an image
    plt.tight_layout()
    plt.savefig('sentiment_distribution.png')  # Save the plot as an image
    plt.close()  # Close the plot to avoid display issues

    # Return the image path for Gradio to display
    return 'sentiment_distribution.png'

# Gradio interface
interface = gr.Interface(fn=plot_sentiment_distribution, inputs=[], outputs="image", title="Telecom Support Feedback Sentiment")

# Launch the interface
interface.launch()